#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##Install pakages


In [ ]:
#Install pakages
%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/requirements.txt --no-cache-dir
import os; os._exit(0)   # ✅ restart automatically

######Restart sesstion after this line at first run to make sure that the new libs are used====

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 210.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 276.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 273.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 168.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 191.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 253.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 342.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 297.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 293.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 361.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 243.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
configs/  notebooks/  requirements-lock.txt  results/           src/
dataset/  README.md   requirements.txt       run_experiment.py  tests/


##Import  libs

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import os
import pandas as pd
import yaml
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import datetime
import warnings
import re
import sweetviz as sv

# ignore warning related to font
warnings.filterwarnings(
    "ignore",
    message="Glyph .* missing from font",  # Pass the pattern as a STRING
    category=UserWarning
)
## Navigate to project location

%pip freeze > requirements-lock.txt



#Helper Code

##Loging

In [3]:

# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_prep.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [4]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file"""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"Loaded config from {config_path}")
    return config

## CDR dataset

In [5]:
def load_cdr(file_path, nrows=None):
    """Load a CDR CSV file with optional row limit (sample mode)."""
    logger.info(f"Loading file: {file_path} with nrows={nrows}")
    return pd.read_csv(file_path, nrows=nrows)


def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
        logger.info(f"Loaded {name} -> {df.shape} from {path}")
    return data


##Genrate profile

In [6]:
import os

def ydata_generate_profile(df, output_file="results/profile.html", mode="minimal", overwrite=True):
    """
    Generate profiling report (ydata-profiling) with safe defaults.
    """
    # Drop very high-cardinality or long-text columns in minimal mode
    if mode == "minimal":
       drop_cols = []
       for col in df.columns:
        if df[col].dtype == "object":  # only apply to categorical/text
            if df[col].nunique() > 5000 or df[col].astype(str).str.len().mean() > 100:
                drop_cols.append(col)
        elif df[col].nunique() <= 1:  # constant numeric (useless)
            drop_cols.append(col)

    if drop_cols:
        print(f"⚠️ Skipping columns for profiling: {drop_cols}")
        df = df.drop(columns=drop_cols)


    # Handle overwrite manually
    if overwrite and os.path.exists(output_file):
        os.remove(output_file)

    profile = ProfileReport(
        df,
        title="Fraud Detection EDA Report",
        explorative=True,
        plot={"wordcloud": False},
        correlations={
            "pearson": {"calculate": mode == "full"},
            "spearman": {"calculate": mode == "full"},
            "kendall": {"calculate": mode == "full"},
            "phi_k": {"calculate": mode == "full"},
            "cramers": {"calculate": mode == "full"},
            "auto": {"calculate": mode == "full"},
        },
        interactions={"continuous": mode == "full"},
        missing_diagrams={"heatmap": mode == "full"},
        duplicates={"calculate": mode == "full"}
    )

    profile.to_file(output_file)  # ✅ no overwrite argument
    print(f"✅ Profiling report saved → {output_file}")
    return output_file


In [7]:
def generate_profile_sweetviz(df, output_file="results/profile_sweetviz.html", target=None):
    """
    Generate a Sweetviz report for df (with optional target column).
    """
    drop_cols = []
    for col in df.columns:
        if df[col].dtype == "object":  # only check high-cardinality categoricals
            if df[col].nunique() > 1000:
                drop_cols.append(col)
        elif df[col].dtype == "datetime64[ns]":  # datetime may break plots
            drop_cols.append(col)

    if drop_cols:
        print(f"⚠️ Dropping columns for Sweetviz: {drop_cols}")
        df = df.drop(columns=drop_cols)

    report = sv.analyze(df, target_feat=target)
    report.show_html(output_file)
    print(f"✅ Sweetviz report generated at {output_file}")
    return output_file



# Load config +  dataset
# ===============================


In [8]:
config = load_config("configs/baseline.yaml")

# Load raw data
data = load_all_data(config)

import datetime

for name, df in data.items():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # -------------------------------
    # Load dataset-specific preprocessing config
    # -------------------------------
    preprocess_cfg = config["preprocessing"].get(name, {})

    numeric_features = preprocess_cfg.get("numeric_features", [])
    categorical_features = preprocess_cfg.get("categorical_features", [])
    datetime_features = preprocess_cfg.get("datetime_features", [])
    target = preprocess_cfg.get("target", None)

    print(f"\n▶ Preprocessing {name}")
    print(f"   numeric: {numeric_features}")
    print(f"   categorical: {categorical_features}")
    print(f"   datetime: {datetime_features}")
    print(f"   target: {target}")

    # -------------------------------
    # Profiling with ydata_profiling
    # -------------------------------
    profiling_mode = config["training"].get("profiling_mode", "minimal")
    ydata_output = f"results/profile_{name}_{timestamp}.html"
    ydata_generate_profile(df, output_file=ydata_output, mode=profiling_mode)

    # -------------------------------
    # Profiling with Sweetviz
    # -------------------------------
    sweetviz_output = f"results/profile_{name}_sweetviz_{timestamp}.html"
    generate_profile_sweetviz(df, output_file=sweetviz_output, target=target)




▶ Preprocessing app
   numeric: ['flow', 'month_id']
   categorical: ['busi_name']
   datetime: []
   target: None
⚠️ Skipping columns for profiling: ['phone_no_m']


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 13.93it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Profiling report saved → results/profile_voc_20250930_044308.html
⚠️ Dropping columns for Sweetviz: ['opposite_no_m', 'start_datetime']


                                             |          | [  0%]   00:00 -> (? left)

Report results/profile_voc_sweetviz_20250930_044308.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
✅ Sweetviz report generated at results/profile_voc_sweetviz_20250930_044308.html
